<a href="https://colab.research.google.com/github/trinade96/Fortiate_Project/blob/master/Fortiate_DST_3_and_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Trina De

##### **Packages Used**

In [0]:
import numpy as np 
import pandas as pd
from datetime import date
import time

##### **Loading the data**

In [2]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


##### **Reading the Data**

In [3]:
missing_values = ["n/a", "na", "--", ' ', "NAN","nan","NA","NaN"]
data = pd.read_excel("/gdrive/My Drive/DATA FOR ALL TEAMS-25MB.xlsx",na_values=missing_values)
data.head()

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,ACQUIRING_COUNTRY_CODE,POS entry Mode,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,Private Data 48,Transaction Category Code,DE48 Data Length,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
0,DE2,DE3,DE4,DE5,DE6,DE7,DE9,DE14,DE16,DE18,DE19,DE22,DE32,DE38,DE39,DE40,DE41,DE42,DE43,DE48,Private Data 48-1,Private Data 48-2,DE49,DE50,DE51,DE61,DE4801,DE4801,DE4803,DE4804,DE4805,DE4806,DE4807,DE4808,DE4809,DE4810
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,NaN,6011,356,???,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000,NaN,NaN,NaN
2,6086941026163002,174000,2000,2000,2000,22-MAR-18 12.28.15,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000,NaN,NaN,NaN
3,6086941026163002,174000,1000,1000,1000,22-MAR-18 14.51.45,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000,NaN,NaN,NaN
4,6086941055683003,174000,4500,28.59,2034.06,22-MAR-18 15.50.10,56874000,01-JUL-23 00.00.00,NaN,6011,144,???,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.4,NaN,NaN,NaN


In [4]:
data2 = data.iloc[1:,:]
data2.head()

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,ACQUIRING_COUNTRY_CODE,POS entry Mode,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,Private Data 48,Transaction Category Code,DE48 Data Length,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,NaN,6011,356,???,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000,NaN,NaN,NaN
2,6086941026163002,174000,2000,2000,2000,22-MAR-18 12.28.15,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000,NaN,NaN,NaN
3,6086941026163002,174000,1000,1000,1000,22-MAR-18 14.51.45,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000,NaN,NaN,NaN
4,6086941055683003,174000,4500,28.59,2034.06,22-MAR-18 15.50.10,56874000,01-JUL-23 00.00.00,NaN,6011,144,???,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.4,NaN,NaN,NaN
5,6086941020177008,004000,499,7.14,499,22-MAR-18 02.35.44,0,01-DEC-22 00.00.00,NaN,5967,840,???,370100,954345,000,201,FN90G440,9590108460,GOOGLE SERDICES \\MOUNTTIN DIEW\94043 \,NaN,NaN,NaN,356,840,356,200111100040,IN1,POS,0797,NaN,NaN,NaN,177495,NaN,NaN,NaN


##### **Changing non NaNs in 'TARGET' column to 'Not Fraud'**

In [5]:
data2['TARGET'] = [x if x == 'Fraud' else 'Not Fraud' for x in list(data2['TARGET'])]
set(data2['TARGET'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


{'Fraud', 'Not Fraud'}

In [6]:
data2.head()

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,ACQUIRING_COUNTRY_CODE,POS entry Mode,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,Private Data 48,Transaction Category Code,DE48 Data Length,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,NaN,6011,356,???,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000,NaN,NaN,Not Fraud
2,6086941026163002,174000,2000,2000,2000,22-MAR-18 12.28.15,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000,NaN,NaN,Not Fraud
3,6086941026163002,174000,1000,1000,1000,22-MAR-18 14.51.45,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000,NaN,NaN,Not Fraud
4,6086941055683003,174000,4500,28.59,2034.06,22-MAR-18 15.50.10,56874000,01-JUL-23 00.00.00,NaN,6011,144,???,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.4,NaN,NaN,Not Fraud
5,6086941020177008,004000,499,7.14,499,22-MAR-18 02.35.44,0,01-DEC-22 00.00.00,NaN,5967,840,???,370100,954345,000,201,FN90G440,9590108460,GOOGLE SERDICES \\MOUNTTIN DIEW\94043 \,NaN,NaN,NaN,356,840,356,200111100040,IN1,POS,0797,NaN,NaN,NaN,177495,NaN,NaN,Not Fraud


In [7]:
data2.shape

(996861, 36)

##### **Removing columns full of NaNs**

In [0]:
indices = []
for i in data2.columns:
  if(pd.isna(data2[i]).sum() == len(data2)):
    indices.append(i)
data2 = data2.drop(indices,axis = 1)

In [9]:
indices

[]

In [10]:
data2.shape

(996861, 36)

##### So we see that none of the columns are full of NaNs, so we retain all 36 of the columns

##### **Dealing with the date type columns**

##### First we will pick out the date type columns. We make this assumption that the columns which are of date type have the term 'DATE' or some form of the word in the name.


In [11]:
names = []
for i in data2.columns:
  names.append(i.upper())
names


['CARD-WALLET-TOKEN ID',
 'PROCESSING_CODE',
 'TRANSACTION_AMOUNT',
 'SETTLEMENT_AMOUNT',
 'BILLING_AMOUNT',
 'TRANSACTION DATE AND TIME',
 'CONVERSION_RATE',
 'EXPIRY DATE',
 'CONVERSION_RATE_DATE',
 'MERCHANT CATEGORY CODE',
 'ACQUIRING_COUNTRY_CODE',
 'POS ENTRY MODE',
 'ACQUIRER_INSTITUTION_CODE',
 'AUTHORIZATION CODE',
 'APPROVAL DECISION',
 'CARD SERVICE CODE',
 'TID',
 'MID',
 'MERCHANT NAME AND ADDRESS',
 'PRIVATE DATA 48',
 'TRANSACTION CATEGORY CODE',
 'DE48 DATA LENGTH',
 'TRANSACTION_CURRENCY',
 'SETTLEMENT_CURRENCY',
 'BILLING_CURRENCY',
 'POS_DATA',
 'INTERFACE ID',
 'TERMINAL TYPE',
 'TOKEN DATA-C1',
 'ADDITIONAL PAYMENT INFO',
 'ECOMMERCE DATA',
 'ECI INDICATOR',
 'REMAINING BALANCE',
 'PRIVATE DATA - ADDITIONAL INFO',
 'TOKEN DATA',
 'TARGET']

In [0]:
data2.columns = names

In [13]:
date_columns = []
for i in names:
  if(i.find('DATE') >= 0):
    date_columns.append(i)
date_columns

['TRANSACTION DATE AND TIME', 'EXPIRY DATE', 'CONVERSION_RATE_DATE']

##### Therefore these are my date columns, we will out checks on them to rule out cases of Fraud.
##### We will compare the transaction date with the current date range and compare it with the conversion rate date which should also be in the vicinity. As for the expiry date we assume that it cannot be before the transaction date and not more than 10 years ahead of the transaction date. Later on we can impose more conditions of similar nature if we want. For this changes will have to be made to the code.

In [14]:
data2['TRANSACTION DATE AND TIME'][1].split(' ')

['22-MAR-18', '09.10.37']

In [15]:
today = date.today()
current_date = today.strftime("%d-%m-%y")
print(current_date)

13-01-20


In [0]:
## creating a dictionary to match the months to their respective serial numbers
month_dict = {'JAN':1,'FEB':2,'MAR':3,'APR':4,'MAY':5,'JUN':6,'JUL':7,'AUG':8,'SEP':9,'OCT':10,'NOV':11,'DEC':12}
## checking is current date is after the transaction date or not because we certainly cannot have a date from the future here
date_1, time_1 = data2['TRANSACTION DATE AND TIME'][1].split(' ')


In [0]:
##splitting the date into month day and year and checking for its validity
def check_transact_date(d):
  today = date.today()
  day, month, year = d.split('-')
  if (int(year)>int(today.strftime("%y"))):
    return 1
  elif(int(year)==int(today.strftime("%y"))) and (month_dict[month]>int(today.strftime("%m"))):
    return 1
  elif(int(year)==int(today.strftime("%y"))) and (month_dict[month]==int(today.strftime("%m"))) and (int(day)>int(today.strftime("%d"))):
    return 1
  else:
    return 0

In [0]:
##checking transaction date and time because it definitely cannot be more than the current time
def check_transact_date_and_time(d,t):
  hours, mins, secs = t.split('.')
  today = date.today()
  day, month, year = d.split('-')
  if(check_transact_date(d)==1): ## checking to see if the date is valid, if not it makes no sense to check time
    return 1
  elif(int(year)==int(today.strftime("%y"))) and (month_dict[month]==int(today.strftime("%m"))) and (int(day)==int(today.strftime("%d"))):
    now = datetime.now()
    if(int(now.strftime("%H"))<int(hours)):
      return 1
    elif(int(now.strftime("%H"))==int(hours)) and (int(now.strftime("%M"))<int(mins)):
      return 1
    elif(int(now.strftime("%H"))==int(hours)) and (int(now.strftime("%M"))==int(mins)) and (int(now.strftime("%S"))<int(secs)):
      return 1
    else:
      return 0
  else:
    return 0

In [19]:
## Now we will find the transactions that are found as frauds on the basis of the above check on transaction date and time
transact_date_fraud_indices = []
data2_temp = data2[pd.isna(data2['TRANSACTION DATE AND TIME'])==False]
data2_temp['new_index'] =  np.arange(1,len(data2_temp)+1,1)
#data2_temp.set_index(index,inplace=True)
data2_temp.set_index(['new_index'], inplace = True) 
start = time.time()
for i in np.arange(1,len(data2_temp),1):
  date_1, time_1 = data2_temp['TRANSACTION DATE AND TIME'][i].split(' ')
  if (check_transact_date_and_time(date_1,time_1)==1):
    transact_date_fraud_indices.append(i)
end = time.time()
print('Time taken to find fraud cases: {}'.format(end-start))
print(len(transact_date_fraud_indices))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Time taken to find fraud cases: 1.212996244430542
0


##### So we can therefore conclude that there are no frauds in terms of the transaction date

##### As for the conversion rate date we can similarly just apply the 'check_transact_date' function to the find the set of valid dates since the rule applied is the same that it cannot be further ahead than the current date. As for how far it can go in the backward direction, it can be decided upon and changed later.

In [20]:
## Now we will find the frauds that were on the basis of the check on conversion dates
conversion_date_fraud_indices = []
data2_temp = data2[pd.isna(data2['CONVERSION_RATE_DATE'])==False]
data2_temp['new_index'] =  np.arange(1,len(data2_temp)+1,1)
#data2_temp.set_index(index,inplace=True)
data2_temp.set_index(['new_index'], inplace = True) 
start = time.time()
for i in np.arange(1,len(data2_temp),1):
  date_1, time_1 = data2_temp['CONVERSION_RATE_DATE'][i].split(' ')
  if (check_transact_date_and_time(date_1, time_1)==1):
    conversion_date_fraud_indices.append(i)
end = time.time()
print('Time taken to find fraud cases: {}'.format(end-start))
print(len(conversion_date_fraud_indices))

Time taken to find fraud cases: 0.03531289100646973
0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##### Therefore there are no frauds on the basis of coversion rate dates either

##### Now we will check for frauds on the basis of the expiry date of the transaction method ususally cards. We note that the expiry date cannot be less than the date of the transaction or even more than 10 years ahead, a slight change in function that checks the transaction dates helps us check this. The time is always 00.00.00 so we no longer have to check the time.

In [0]:
def check_expiry_date(d1, d2): ## Here d1 is the expiry date and d2 is the transaction date
  day1, month1, year1 = d1.split('-')
  day2, month2, year2 = d2.split('-')
  if (int(year1)<int(year2)) or (int(year1)>int(year2)+15):
    return 1
  elif(int(year1)==int(year2)) and (month_dict[month1]<month_dict[month2]):
    return 1
  elif(int(year1)==int(year2)) and (month_dict[month1]==month_dict[month2]) and (int(day1)<int(day2)):
    return 1
  else:
    return 0

In [22]:
check_expiry_date('23-SEP-49','25-DEC-20')

1

##### Here the error is that the expiry date is more than 15 years ahead of the transaction date which is usually not the case in real life.

In [24]:
## Now we will find the frauds that were on the basis of expiry dates
expiry_date_fraud_indices = []
data2_temp = data2[pd.isna(data2['EXPIRY DATE'])==False]
data2_temp = data2_temp[pd.isna(data2_temp['TRANSACTION DATE AND TIME'])==False]
data2_temp['new_index'] =  np.arange(1,len(data2_temp)+1,1)
#data2_temp.set_index(index,inplace=True)
data2_temp.set_index(['new_index'], inplace = True) 
start = time.time()
for i in np.arange(1,len(data2_temp),1):
  date_1, time_1 = data2_temp['EXPIRY DATE'][i].split(' ')
  date_2, time_2 = data2_temp['TRANSACTION DATE AND TIME'][i].split(' ')
  if (check_expiry_date(date_1,date_2)==1):
    expiry_date_fraud_indices.append(i)
end = time.time()
print('Time taken to find fraud cases: {}'.format(end-start))
print(len(expiry_date_fraud_indices))

Time taken to find fraud cases: 1.0639889240264893
0


##### Unfortunately there are no frauds on the basis of expiry dates either. We can modify the code for other more stringent conditions in the future. Moving forward we will have all supervised learning algorithms that will look at cases of fraud to identify patterns.


##### **ROUGH WORK**

In [0]:
data2.reset_index(inplace=True,drop=True)
data2_temp.head()

,CARD-WALLET-TOKEN ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,TRANSACTION DATE AND TIME,CONVERSION_RATE,EXPIRY DATE,CONVERSION_RATE_DATE,MERCHANT CATEGORY CODE,ACQUIRING_COUNTRY_CODE,POS ENTRY MODE,ACQUIRER_INSTITUTION_CODE,AUTHORIZATION CODE,APPROVAL DECISION,CARD SERVICE CODE,TID,MID,MERCHANT NAME AND ADDRESS,PRIVATE DATA 48,TRANSACTION CATEGORY CODE,DE48 DATA LENGTH,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,INTERFACE ID,TERMINAL TYPE,TOKEN DATA-C1,ADDITIONAL PAYMENT INFO,ECOMMERCE DATA,ECI INDICATOR,REMAINING BALANCE,PRIVATE DATA - ADDITIONAL INFO,TOKEN DATA,TARGET
664,6086941023394006,004000,182.03,12518.2,12518.2,22-MAR-18 03.34.13,1,01-DEC-22 00.00.00,22-JUL-18 00.00.00,7011,834,???,370100,??????,117,201,2S90GE04,9734114549,KISIWT ON THE BE301579\\ZTNZIBTR\\\834,NaN,NaN,NaN,840,356,356,510101513301,IN1,POS,0784,NaN,NaN,NaN,NaN,NaN,NaN,Fraud
41769,6854591729122000,000000,3500,50.91,3500,22-MAR-18 01.09.34,0,01-MAY-23 00.00.00,21-JUL-18 00.00.00,7011,356,051,007064,??????,100,206,90106669,THE ONTIME HOTE,THE ONTIME HOTEL KALYAN IND,???,?,??,356,840,356,310101500001,IN9,POS,0153,MPLP2FNP5,NaN,NaN,NaN,0000000000900356400059,NaN,Not Fraud
41784,6854593019894000,000000,49000,712.73,49000,22-MAR-18 10.39.43,0,01-MAR-23 00.00.00,21-JUL-18 00.00.00,5812,356,051,017014,??????,100,226,42263068,42MS00000286253,MSW*DICTORY ENTERPRISE Bangalore IND,????,?,???,356,840,356,310101500001,IN9,POS,NaN,MPL2C5RJC,NaN,NaN,NaN,0000000009500356560028,NaN,Fraud
41785,6854592107326000,171000,5000,72.73,5000,22-MAR-18 10.41.17,0,01-JUN-23 00.00.00,21-JUL-18 00.00.00,6011,356,901,088792,??????,100,206,90328012,SBI BHANDARA AT,SBI BHANDARA ATM 12 THARSA IND,???,?,??,356,840,356,210301200001,IN9,POS,NaN,MPL1UVCLA,NaN,NaN,NaN,1010000001200356441104,NaN,Fraud
41786,6854592107326000,171000,10000,145.45,10000,22-MAR-18 10.41.48,0,01-JUN-23 00.00.00,21-JUL-18 00.00.00,6011,356,901,088792,??????,100,206,90328012,SBI BHANDARA AT,SBI BHANDARA ATM 12 THARSA IND,???,?,??,356,840,356,210301200001,IN9,POS,NaN,MPL1VKDDS,NaN,NaN,NaN,1010000001200356441104,NaN,Fraud


In [0]:
data2_temp['CONVERSION_RATE_DATE'][664]

'22-JUL-18 00.00.00'

In [0]:
pd.isna(data2['CONVERSION_RATE_DATE']).sum()

995661

In [0]:
('abs123').upper()

'ABS123'

In [0]:
'geeks'.find('e')

1

In [0]:
check_transact_date(date_1)

0

In [0]:
check_transact_date_and_time(date_1,time_1)

0

In [0]:
check_transact_date_and_time(d='13-JAN-20',t='22.30.10')

1

In [0]:
func = lambda s: s[:2].upper() + s[2:].lower() if s else ''
date, time = data2['TRANSACTION DATE AND TIME'][1].split(' ')
func(date)

'22-mar-18'

In [0]:


# dd/mm/YY
d1 = today.strftime("%d/%m/%Y")
print("d1 =", d1)
# Textual month, day and year	
d2 = today.strftime("%B %d, %Y")
print("d2 =", d2)
# mm/dd/y
d3 = today.strftime("%m/%d/%y")
print("d3 =", d3)
# Month abbreviation, day and year	
d4 = today.strftime("%b-%d-%y")
print("d4 =", d4)

d1 = 13/01/2020
d2 = January 13, 2020
d3 = 01/13/20
d4 = Jan-13-20


In [0]:
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

now = 2020-01-13 14:23:54.203888
date and time = 13/01/2020 14:23:54


In [0]:
data2_temp = data2[pd.isna(data2['TRANSACTION DATE AND TIME'])==False]
data2_temp.head()

,CARD-WALLET-TOKEN ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,TRANSACTION DATE AND TIME,CONVERSION_RATE,EXPIRY DATE,CONVERSION_RATE_DATE,MERCHANT CATEGORY CODE,ACQUIRING_COUNTRY_CODE,POS ENTRY MODE,ACQUIRER_INSTITUTION_CODE,AUTHORIZATION CODE,APPROVAL DECISION,CARD SERVICE CODE,TID,MID,MERCHANT NAME AND ADDRESS,PRIVATE DATA 48,TRANSACTION CATEGORY CODE,DE48 DATA LENGTH,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,INTERFACE ID,TERMINAL TYPE,TOKEN DATA-C1,ADDITIONAL PAYMENT INFO,ECOMMERCE DATA,ECI INDICATOR,REMAINING BALANCE,PRIVATE DATA - ADDITIONAL INFO,TOKEN DATA,TARGET
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,NaN,6011,356,???,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000,NaN,NaN,Not Fraud
2,6086941026163002,174000,2000,2000,2000,22-MAR-18 12.28.15,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000,NaN,NaN,Not Fraud
3,6086941026163002,174000,1000,1000,1000,22-MAR-18 14.51.45,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000,NaN,NaN,Not Fraud
4,6086941055683003,174000,4500,28.59,2034.06,22-MAR-18 15.50.10,56874000,01-JUL-23 00.00.00,NaN,6011,144,???,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.4,NaN,NaN,Not Fraud
5,6086941020177008,004000,499,7.14,499,22-MAR-18 02.35.44,0,01-DEC-22 00.00.00,NaN,5967,840,???,370100,954345,000,201,FN90G440,9590108460,GOOGLE SERDICES \\MOUNTTIN DIEW\94043 \,NaN,NaN,NaN,356,840,356,200111100040,IN1,POS,0797,NaN,NaN,NaN,177495,NaN,NaN,Not Fraud
